# Jacobian Pseudo Inverse Controller
ToDo: Add theory of operation

## Run In Simulation

In [1]:
import time
import numpy as np
from FR3Py.controllers.waypoint_controller import WaypointController
from FR3Py.envs.pybullet import FR3Sim

robot = FR3Sim(render_mode='human', mode='velocity', Ts=1/1000)
robot.reset()
controller = WaypointController(kp=2)
while True:
    q, dq = robot.get_state()
    dq_cmd = controller.compute(q, dq)
    robot.step(dq_cmd)
    time.sleep(1/1000)

AttributeError: 'NoneType' object has no attribute '__dict__'

## Run With Real Robot

In [1]:
import sys 
sys.path.append("/usr/local/lib/python3.8/site-packages")
from FR3Py.interfaces import FR3Real

robot = FR3Real()

Interface Running...


Test the interface by reading the robot joints:

In [ ]:
robot.get_state()

In [28]:
state = robot.get_state()
q, dq = state['q'], state['dq']

print(q, dq)

[-9.01207363e-05 -7.85094045e-01  1.43095362e-04 -2.35606960e+00
 -3.94108067e-04  1.57092448e+00  7.85301659e-01  0.00000000e+00
  0.00000000e+00] [-3.31773997e-04  1.09744403e-03 -9.75158057e-04 -3.04230352e-04
  3.26794884e-04 -1.27994316e-03 -9.72842622e-05  0.00000000e+00
  0.00000000e+00]


In [29]:
import time
import numpy as np
from FR3Py.controllers.waypoint_controller import WaypointController
controller = WaypointController(kp=1.5)
# Read the initila pose of the robot
time.sleep(1)
state = robot.get_state()

q, dq = state['q'], state['dq']
p0 = controller.robot.getInfo(q,dq)['P_EE']
R0 = controller.robot.getInfo(q,dq)['R_EE']
T0 = np.vstack([np.hstack([R0, p0.reshape(3,1)]), np.array([0,0,0,1])])
start_time = time.time()
while True:
    time.sleep(0.01)
    t = time.time() - start_time
    x = 0.1 * np.sin(t)
    T= T0 @ np.vstack([np.hstack([np.eye(3), np.array([0,0,x]).reshape(3,1)]), np.array([0,0,0,1])])
    state = robot.get_state()
    q, dq = state['q'], state['dq']
    cmd = controller.compute(q, dq, T_cmd=T)
    robot.send_joint_command(cmd[0:7])

KeyboardInterrupt: 